In [1]:
from typing import List, Union
from tqdm import tqdm
import string
import datetime
from dateutil import parser
import multiprocessing as mp

import pandas as pd

import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.model_selection import train_test_split


import pyarrow as pa
import pyarrow.dataset as ds

# import boto3

In [2]:
# !pip uninstall tqdm -Y
# !pip install tqdm==2.0.0

  Using cached tqdm-2.0.0-py2.py3-none-any.whl (16 kB)
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.1
    Uninstalling tqdm-4.64.1:
      Successfully uninstalled tqdm-4.64.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.24.0 requires tqdm>=4.27, but you have tqdm 2.0.0 which is incompatible.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.0.2 which is incompatible.
panel 0.13.1 requires tqdm>=4.48.0, but you have tqdm 2.0.0 which is incompatible.
evaluate 0.3.0 requires tqdm>=4.62.1, but you have tqdm 2.0.0 which is incompatible.
datasets 2.3.2 requires tqdm>=4.62.1, but you have tqdm 2.0.0 which is incompatible.
anaconda-client 1.11.0 requires tqdm>=4.56.0, but you have tqdm 2.0.0 which is incompatible.


In [3]:
!python -m pip install git+https://github.com/huggingface/setfit.git -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.0.2 which is incompatible.


In [17]:
# !pip install setfit -q

In [2]:
# s3 = boto3.client('s3')
# obj = s3.get_object(Bucket = 'mlops-capstone-project',Key = 'reduced_email.csv')

df = pd.read_csv('../data/sagemaker-testing/Parsed/reduced_email.csv')
# df = pd.read_csv(obj['Body'])

In [3]:
df.head()

,Message-ID,Date,From,To,Subject,Cc,Mime-Version,Content-Type,Content-Transfer-Encoding,Bcc,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,body,spam,body_transformed
0,<28435830.1075860750114.JavaMail.evans@thyme>,2002-01-22 17:22:17,"[""enerfaxdaily@enerfax.com""]","[""darrell.schoolcraft@enron.com""]",[enerfaxdaily] Natural Gas Futures Driven Lowe...,NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,<enerfaxdaily@enerfax.com>@ENRON,"Schoolcraft, Darrell </O=ENRON/OU=NA/CN=RECIPI...",NaN,NaN,"\Darrell_Schoolcraft_Mar2002\Schoolcraft, Darr...",Schoolcraft-D,dschool (Non-Privileged).pst,"If your do not see our email in 15 seconds, pl...",1,email 15 second http://www.enerfaxdaily.com to...
1,<31088330.1075860887438.JavaMail.evans@thyme>,2004-03-02 18:14:47,"[""hotwecash@lists.adversend.com""]","[""m..presto@enron.com""]",Your Opinion Counts...Win $1000!,NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,HotWebCash Newsletter <hotwebcash@lists.advers...,"Presto, Kevin M. </O=ENRON/OU=NA/CN=RECIPIENTS...",NaN,NaN,"\Kevin_Presto_Mar2002_1\Presto, Kevin M.\Junk ...",Presto-K,kpresto (Non-Privileged).pst,$ * $ * $ * $ * HOTWEBCASH * $ * $ * $ * $Your...,1,hotwebcash opinion count ... reward It!Join NP...
2,<8829953.1075860887541.JavaMail.evans@thyme>,2004-03-02 18:14:47,"[""hotwecash@lists.adversend.com""]","[""m..presto@enron.com""]",LOSE POUNDS GUARANTEED!,NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,HotWebCash Newsletter <hotwebcash@lists.advers...,"Presto, Kevin M. </O=ENRON/OU=NA/CN=RECIPIENTS...",NaN,NaN,"\Kevin_Presto_Mar2002_1\Presto, Kevin M.\Junk ...",Presto-K,kpresto (Non-Privileged).pst,$ * $ * $ * $ * $ HOTWEBCASH $ * $ * $ * $ * $...,1,hotwebcash program tailor specific needs!eat F...
3,<26855977.1075860939101.JavaMail.evans@thyme>,2002-01-23 11:36:44,"[""jjill@clickaction.net""]","[""lindy.donoho@enron.com""]",Great spring values from jjill.com,NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,J. Jill <jjill@clickaction.net>,"Donoho, Lindy </O=ENRON/OU=NA/CN=RECIPIENTS/CN...",NaN,NaN,"\Lindy_Donoho_Mar2002_1\Donoho, Lindy\Inbox\Ju...",Donoho-L,ldonoho (Non-Privileged).pst,<! ><! ><! ATTENTION!><! You are reading this ...,1,attention read message mail reader display HTM...
4,<25503194.1075860887462.JavaMail.evans@thyme>,2004-03-02 18:14:47,"[""hotwecash@lists.adversend.com""]","[""m..presto@enron.com""]",Someone is searching for YOU!,NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,HotWebCash Newsletter <hotwebcash@lists.advers...,"Presto, Kevin M. </O=ENRON/OU=NA/CN=RECIPIENTS...",NaN,NaN,"\Kevin_Presto_Mar2002_1\Presto, Kevin M.\Junk ...",Presto-K,kpresto (Non-Privileged).pst,$ * $ * $ * $ * HOTWEBCASH * $ * $ * $ * $Find...,1,hotwebcash find match ... Click http://r1.adve...


In [4]:
spam_df = df[df['spam'] == 1].sample(8)
notspam_df = df[df['spam'] == 0].sample(8)

In [5]:
reduced_df = pd.concat([spam_df, notspam_df]).reset_index(drop=True)

In [6]:
reduced_df

,Message-ID,Date,From,To,Subject,Cc,Mime-Version,Content-Type,Content-Transfer-Encoding,Bcc,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,body,spam,body_transformed
0,<28026099.1075860887409.JavaMail.evans@thyme>,2004-03-02 18:14:47,"[""hotwecash@lists.adversend.com""]","[""m..presto@enron.com""]",Need Part Time Work!?,NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,HotWebCash Newsletter <hotwebcash@lists.advers...,"Presto, Kevin M. </O=ENRON/OU=NA/CN=RECIPIENTS...",NaN,NaN,"\Kevin_Presto_Mar2002_1\Presto, Kevin M.\Junk ...",Presto-K,kpresto (Non-Privileged).pst,$ * $ * $ * $ * HOTWEBCASH * $ * $ * $ * $Find...,1,hotwebcash find home business s right you!!CLI...
1,<22393351.1075855176790.JavaMail.evans@thyme>,2001-12-18 16:53:57,"[""oa@energy.ca.gov""]","[""newsreleases@energy.ca.gov"", ""henrietta@ener...",[NEWSRELEASES-LIST] News Release - Energy Comm...,NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,boba <boba@energy.ca.gov>,"newsreleases@energy.ca.gov, henrietta@energy.c...",NaN,NaN,"\Jane_Tholt_Jan2002_1\Tholt, Jane M.\Inbox",Tholt-J,jtholt (Non-Privileged).pst,----------------------------------------------...,1,----------------------------------------------...
2,<19192454.1075860887517.JavaMail.evans@thyme>,2004-03-02 18:14:47,"[""hotwecash@lists.adversend.com""]","[""m..presto@enron.com""]","This email could of been worth $10,000!",NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,HotWebCash Newsletter <hotwebcash@lists.advers...,"Presto, Kevin M. </O=ENRON/OU=NA/CN=RECIPIENTS...",NaN,NaN,"\Kevin_Presto_Mar2002_1\Presto, Kevin M.\Junk ...",Presto-K,kpresto (Non-Privileged).pst,$ * $ * $ * $ * HOTWEBCASH * $ * $ * $ * $iWon...,1,hotwebcash iWon web portal like Yahoo givesawa...
3,<17297268.1075860950992.JavaMail.evans@thyme>,2002-03-21 10:51:25,"[""talots@clickaction.net""]","[""lindy.donoho@enron.com""]",Talbots - Our Classic Mid-Season Sale,NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,talbots.com <talbots@clickaction.net>,"Donoho, Lindy </O=ENRON/OU=NA/CN=RECIPIENTS/CN...",NaN,NaN,"\Lindy_Donoho_Mar2002_1\Donoho, Lindy\Inbox\Ju...",Donoho-L,ldonoho (Non-Privileged).pst,<! ><! ><! ATTENTION!><! You are reading this ...,1,attention read message mail reader display HTM...
4,<1165601.1075860887380.JavaMail.evans@thyme>,2004-03-02 18:14:47,"[""hotwecash@lists.adversend.com""]","[""m..presto@enron.com""]",Someone is searching for YOU!,NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,HotWebCash Newsletter <hotwebcash@lists.advers...,"Presto, Kevin M. </O=ENRON/OU=NA/CN=RECIPIENTS...",NaN,NaN,"\Kevin_Presto_Mar2002_1\Presto, Kevin M.\Junk ...",Presto-K,kpresto (Non-Privileged).pst,$ * $ * $ * $ * HOTWEBCASH * $ * $ * $ * $Find...,1,hotwebcash find match ... Click http://r1.adve...
5,<31088330.1075860887438.JavaMail.evans@thyme>,2004-03-02 18:14:47,"[""hotwecash@lists.adversend.com""]","[""m..presto@enron.com""]",Your Opinion Counts...Win $1000!,NaN,1.0,text/plain; charset=us-ascii,7bit,NaN,HotWebCash Newsletter <hotwebcash@lists.advers...,"Presto, Kevin M. </O=ENRON/OU=NA/CN=RECIPIENTS...",NaN,NaN,"\Kevin_Presto_Mar2002_1\Presto, Kevin M.\Junk ...",Presto-K,kpresto (Non-Privileged).pst,$ * $ * $ * $ * HOTWEBCASH * $ * $ * $ * $Your...,1,hotwebcash opinion count ... reward It!Join NP...
6,<13358041.1075859840113.JavaMail.evans@thyme>,2001-04-19 04:40:00,"[""utiliconnews.au@terrapinn.com""]","[""mhaedic@ect.enron.com""]",SA Government looks to cushion price hike,NaN,1.0,text/plain; charset=ANSI_X3.4-1968,quoted-printable,NaN,The Utility Forum <utiliconnews.au@terrapinn.com>,Mr Mark E Haedicke <mhaedic@ect.enron.com>,NaN,NaN,\Mark_Haedicke_Jun2001\Notes Folders\Notes inbox,Haedicke-M,mhaedic.nsf,UTILICON AUSTRALIA 2001_______________________...,1,UTILICON AUSTRALIA 2001_______________________...
7,<27338117.1075860887569.JavaMail.evans@thyme>,2004-03-02 18:14:47,"[""hotwecash@lists.adversend.com""]","[""m..presto@enron.com""]",confirmation #87717,NaN,1.0,text/plain; charset=ANSI_X3.4-1968,7bit,NaN,HotWebCash Newsletter <hotwebcash@l

In [7]:
from datasets import Dataset

/Users/luis.morales/Desktop/MLOpsBootcamp/MLOpsCapstoneProject/mlopsenv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
train_df, test_df = train_test_split(reduced_df, test_size=0.5)

In [12]:
test_df.spam

3     1
12    0
5     1
8     0
15    0
0     1
13    0
2     1
Name: spam, dtype: int64

In [13]:
from sentence_transformers.losses import CosineSimilarityLoss

from setfit import SetFitModel, SetFitTrainer, sample_dataset

There was a problem when trying to write in your cache folder (/Users/luis.morales/.cache/huggingface/hub). You should set the environment variable TRANSFORMERS_CACHE to a writable directory.


In [14]:
# Simulate the few-shot regime by sampling 8 examples per class
train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(test_df)

In [16]:
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2", cache_dir='~/Desktop/MLOpsBootcamp/MLOpsCapstoneProject')

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [17]:
# Create trainer
trainer = SetFitTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss_class=CosineSimilarityLoss,
    metric="accuracy",
    batch_size=16,
    num_iterations=20, 
    num_epochs=1, 
    column_mapping={"body_transformed": "text", "spam": "label"} 
    
)

In [18]:
trainer.train()

Applying column mapping to training dataset
***** Running training *****
  Num examples = 320
  Num epochs = 1
  Total optimization steps = 20
  Total train batch size = 16
Epoch:   0%|          | 0/1 [00:00<?, ?it/s]